In [56]:
import os
import pandas as pd
import sys
import torch

sys.path.append("../")
# sys.path.append("../hms_pipeline")

from hms_pipeline.spec_dataset import SpecDataset, infer_spec_dataset
from hms_pipeline.eeg_model import SpecModel, SpecVitModel
from hms_pipeline.kaggle_kl_div import eval_subm, eval_subm_g10, team_eval

from hms_pipeline.eeg_model import EegInferModule


In [57]:
gt_df_path = "/home/maxc/workspace/kaggle-hms/folds/all_targets.csv"

In [58]:
result_dir = "/home/maxc/workspace/kaggle-hms/oof_results/exp"

# list all csv files in the result directory
csv_files = [f for f in os.listdir(result_dir) if f.endswith(".csv")]
print(csv_files)


['hmsv6-maxvitt-w025-3imgs-kl1-bandclip-avgmax.csv', 'lrflip_hmsv6-maxvits-4imgs-w025-kl1-512_logit.csv', 'lrflip_hmsv6-convnexts-w025-4imgs-kl1-bandclip-12s-large.csv', 'hmsv6-convnexts-w025-3imgs-kl1-bandclip-20s_logit.csv', 'hmsv6-maxvitt-4imgs-w025-kl1-avgmax_logit.csv', 'lrflip_hmsv6-maxvitt-4imgs-w025-kl1-avgmax_logit.csv', 'hmsv6-maxvits-4imgs-w025-kl1-512_logit.csv', 'lrflip_hmsv6-maxvits-4imgs-w025-kl1-512.csv', 'lrflip_hmsv6-maxvits-w025-3imgs-kl1-bandclip_logit.csv', 'hmsv6-maxvitt-4imgs-w025-kl1-avgmax.csv', 'lrflip_hmsv6-maxvitt-w025-3imgs-kl1-bandclip-avgmax.csv', 'hmsv6-maxvitt-w025-3imgs-kl1-bandclip-avgmax_logit.csv', 'hmsv6-convnexts-w025-4imgs-kl1-bandclip-12s-large_logit.csv', 'lrflip_hmsv6-convnexts-w025-4imgs-kl1-bandclip-12s-large_logit.csv', 'hmsv6-convnexts-w025-3imgs-kl1-bandclip-20s.csv', 'lrflip_hmsv6-maxvits-w025-3imgs-kl1-bandclip.csv', 'lrflip_hmsv6-maxvitt-4imgs-w025-kl1-avgmax.csv', 'lrflip_hmsv6-convnexts-w025-3imgs-kl1-bandclip-20s_logit.csv', 'hmsv6-

In [59]:
logits_csv_files = [f for f in csv_files if "logit" in f]
print(logits_csv_files)
print(len(logits_csv_files))
# extra non-logit files
non_logits_csv_files = [f for f in csv_files if f not in logits_csv_files]
print(non_logits_csv_files)
print(len(non_logits_csv_files))

['lrflip_hmsv6-maxvits-4imgs-w025-kl1-512_logit.csv', 'hmsv6-convnexts-w025-3imgs-kl1-bandclip-20s_logit.csv', 'hmsv6-maxvitt-4imgs-w025-kl1-avgmax_logit.csv', 'lrflip_hmsv6-maxvitt-4imgs-w025-kl1-avgmax_logit.csv', 'hmsv6-maxvits-4imgs-w025-kl1-512_logit.csv', 'lrflip_hmsv6-maxvits-w025-3imgs-kl1-bandclip_logit.csv', 'hmsv6-maxvitt-w025-3imgs-kl1-bandclip-avgmax_logit.csv', 'hmsv6-convnexts-w025-4imgs-kl1-bandclip-12s-large_logit.csv', 'lrflip_hmsv6-convnexts-w025-4imgs-kl1-bandclip-12s-large_logit.csv', 'lrflip_hmsv6-convnexts-w025-3imgs-kl1-bandclip-20s_logit.csv', 'hmsv6-maxvits-w025-3imgs-kl1-bandclip_logit.csv', 'lrflip_hmsv6-maxvitt-w025-3imgs-kl1-bandclip-avgmax_logit.csv']
12
['hmsv6-maxvitt-w025-3imgs-kl1-bandclip-avgmax.csv', 'lrflip_hmsv6-convnexts-w025-4imgs-kl1-bandclip-12s-large.csv', 'lrflip_hmsv6-maxvits-4imgs-w025-kl1-512.csv', 'hmsv6-maxvitt-4imgs-w025-kl1-avgmax.csv', 'lrflip_hmsv6-maxvitt-w025-3imgs-kl1-bandclip-avgmax.csv', 'hmsv6-convnexts-w025-3imgs-kl1-bandclip

In [60]:
# ensemble non-logit files
targets = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
for i, f in enumerate(non_logits_csv_files):
    if i == 0:
        pred_df = pd.read_csv(os.path.join(result_dir, f))
    else:
        new_df = pd.read_csv(os.path.join(result_dir, f))
        pred_df[targets] += new_df[targets]

for i, f in enumerate(logits_csv_files):
    if i == 0:
        logits_df = pd.read_csv(os.path.join(result_dir, f))
    else:
        new_df = pd.read_csv(os.path.join(result_dir, f))
        logits_df[targets] += new_df[targets]

logits_df[targets] /= len(logits_csv_files)
pred_df[targets] /= len(non_logits_csv_files)

pred_df.head()

,eeg_id,eeg_sub_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1001487592,0,0.016213,0.224196,0.717423,0.003630,0.007795,0.030743
1,1001487592,1,0.016428,0.203653,0.737474,0.003449,0.007745,0.031251
2,1001487592,2,0.016179,0.226115,0.717429,0.003012,0.007280,0.029984
3,1001487592,3,0.017663,0.259043,0.658045,0.005403,0.015179,0.044666
4,1003458521,0,0.498595,0.169972,0.002912,0.125302,0.006022,0.197198


In [61]:
logits_df.head()

,eeg_id,eeg_sub_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1001487592,0,-0.562442,2.063645,3.262354,-2.147762,-1.336087,0.072426
1,1001487592,1,-0.523271,2.020489,3.331386,-2.224522,-1.373693,0.111637
2,1001487592,2,-0.531257,2.129468,3.311314,-2.266996,-1.409494,0.092604
3,1001487592,3,-0.736283,1.953989,2.922167,-2.007312,-1.016706,0.191345
4,1003458521,0,2.360499,1.262834,-2.825444,0.930483,-2.038983,1.450357


In [62]:
logits_pred_df = logits_df.copy()
logits_tensor = torch.tensor(logits_df[targets].values)
logits_pred_df.to_csv("maxc_ensemble_logits.csv", index=False)

logits_pred_df[targets] = torch.nn.functional.softmax(logits_tensor, dim=1).numpy()
logits_pred_df.head()

,eeg_id,eeg_sub_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1001487592,0,0.015824,0.218678,0.725100,0.003242,0.007300,0.029856
1,1001487592,1,0.015764,0.200633,0.744248,0.002876,0.006735,0.029744
2,1001487592,2,0.015526,0.222119,0.724195,0.002737,0.006451,0.028973
3,1001487592,3,0.017211,0.253620,0.667818,0.004829,0.013003,0.043519
4,1003458521,0,0.501681,0.167385,0.002807,0.120055,0.006162,0.201910


In [63]:
print(eval_subm(pred_df, gt_df=pd.read_csv(gt_df_path)))
print(eval_subm_g10(pred_df, gt_df=pd.read_csv(gt_df_path)))

pred_df.to_csv("maxc_ensemble_preds.csv", index=False)


(106800, 7) (106800, 7) (106800, 7)
0.5724647766649815
(106800, 7) (39946, 7) (39946, 7)
0.25932470603395225


In [64]:
team_eval(pred_df)
team_eval(logits_pred_df)

(17089, 7) (17089, 7)
(5939, 7) (5939, 7)
0.4560954483385081 0.22699059184959083
(17089, 7) (17089, 7)
(5939, 7) (5939, 7)
0.4570175905629792 0.2256930019561901
